# Assignment 4

test if the distribution of 

1) trip duration of bikers that ride during the day vs night

2) age of bikers for trips originating in Manhattan and in Brooklyn

are different. Use 3 tests: KS, Pearson's, Spearman's. 

Use the scipy.stats functions scipy.stats.ks_2samp, scipy.stats.pearsonr, scipy.stats.spearmanr. 

For the KS do the test with the entire dataset and with a subset 200 times smaller

Choose a single significant threshold for the whole exercise. 

For each test phrase the Null Hypothesis in words.

Describe the return of the scipy function you use in each case.

State the result in terms of rejection of the Null.

In [51]:
# my usual imports and setups
import pylab as pl
import pandas as pd
import numpy as np
import datetime as dt
%pylab inline

import os
#this makes my plots pretty! but it is totally not mandatory to do it
import json


Populating the interactive namespace from numpy and matplotlib


In [4]:
puidata = os.getenv("PUIDATA")
if not puidata or puidata == '/nfshome/sjs909/PUIdata':
    puidata = '/nfshome/sjs909/PUIDATA'

os.environ["PUIDATA"] = puidata
os.environ["PUIDATA"]

'/nfshome/sjs909/PUIDATA'

In [ ]:
# Read in the Data

In [30]:
#read in the data from fall season and winter season
jan18 = pd.read_csv("%s/JC-201801-citibike-tripdata.csv.zip"%os.getenv("PUIDATA"))
sep18 = pd.read_csv("%s/JC-201809-citibike-tripdata.csv.zip"%os.getenv("PUIDATA"))


Concatenate the two datasets together

In [31]:
df = pd.concat([jan18,sep18],axis=0)

In [14]:
df.tail()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
39017,985,2018-09-29 09:02:31.3090,2018-09-29 09:18:56.6570,3694,Jackson Square,40.71113,-74.0789,3211,Newark Ave,40.721525,-74.046305,29492,Subscriber,1991,2
39018,1595,2018-09-30 09:36:44.8650,2018-09-30 10:03:19.9750,3694,Jackson Square,40.71113,-74.0789,3183,Exchange Place,40.716247,-74.033459,29252,Subscriber,1987,1
39019,1996,2018-09-30 11:15:46.8100,2018-09-30 11:49:03.1200,3694,Jackson Square,40.71113,-74.0789,3186,Grove St PATH,40.719586,-74.043117,29122,Customer,1987,2
39020,1803,2018-09-30 11:16:37.0760,2018-09-30 11:46:40.6460,3694,Jackson Square,40.71113,-74.0789,3273,Manila & 1st,40.721651,-74.042884,29251,Customer,1974,1
39021,779,2018-09-30 13:49:47.3930,2018-09-30 14:02:46.7200,3694,Jackson Square,40.71113,-74.0789,3192,Liberty Light Rail,40.711242,-74.055701,29653,Customer,1969,0


In [32]:
df['date'] = pd.to_datetime(df['starttime'])

In [26]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,Time
0,932,2018-01-01 02:06:17.5410,2018-01-01 02:21:50.0270,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31929,Subscriber,1992,1,2018-01-01 02:06:17.541,02:06:17.541000
1,550,2018-01-01 12:06:18.0390,2018-01-01 12:15:28.4430,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31845,Subscriber,1969,2,2018-01-01 12:06:18.039,12:06:18.039000
2,510,2018-01-01 12:06:56.9780,2018-01-01 12:15:27.8100,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31708,Subscriber,1946,1,2018-01-01 12:06:56.978,12:06:56.978000
3,354,2018-01-01 14:53:10.1860,2018-01-01 14:59:05.0960,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,31697,Subscriber,1994,1,2018-01-01 14:53:10.186,14:53:10.186000
4,250,2018-01-01 17:34:30.1920,2018-01-01 17:38:40.9840,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,-74.034234,31861,Subscriber,1991,1,2018-01-01 17:34:30.192,17:34:30.192000


In [33]:
df['Time'] = [datetime.datetime.time(d) for d in df['date']]

In [71]:
df.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender', 'date', 'Time'],
      dtype='object')

# Split By Category

Looking at the trip duration of bikers that ride during the day vs night.

H0: there is no statistical difference in the trip duration of day and night riders <br>

$\alpha = 0.05 $

In [69]:
df_roh = df[['starttime','tripduration']]
df_roh.head()

,starttime,tripduration
0,2018-01-01 02:06:17.5410,932
1,2018-01-01 12:06:18.0390,550
2,2018-01-01 12:06:56.9780,510
3,2018-01-01 14:53:10.1860,354
4,2018-01-01 17:34:30.1920,250


In [28]:
# Drop columns we don't need
df_dayNight = df[['Time','tripduration']]

In [102]:
df_dayNight['tripduration'][2]

2     510
2    1181
Name: tripduration, dtype: int64

In [73]:
df_dayNight.head()

,Time,tripduration
0,02:06:17.541000,932
1,12:06:18.039000,550
2,12:06:56.978000,510
3,14:53:10.186000,354
4,17:34:30.192000,250


In [96]:
df_dayNight.shape

(51699, 2)

In [98]:
(df_dayNight['Time'][0] > datetime.time(12))

0    False
0    False
Name: Time, dtype: bool

In [95]:
for i in range(10):
    if (df_dayNight['Time'][i] < datetime.time(12)) == True:
       print('Day' )

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [77]:
for index, row in df_dayNight.iterrows():
    df_dayNight['Time'][index] < datetime.time(12)


KeyboardInterrupt: 

Lets define night as 8:01 PM until 5:59 AM and day as 6:00 AM to 8:00 PM and split the data accordingly
        

In [46]:
by_time = df.groupby(df.index.time).mean()

AttributeError: 'Int64Index' object has no attribute 'time'

In [63]:
df['Time'][1] < pd.datetime(20:00:00)

SyntaxError: invalid syntax (<ipython-input-63-d8925c1a269d>, line 1)

In [53]:
idx

DatetimeIndex(['2018-10-04 00:00:00', '2018-10-04 01:00:00',
               '2018-10-04 02:00:00', '2018-10-04 03:00:00',
               '2018-10-04 04:00:00', '2018-10-04 05:00:00',
               '2018-10-04 06:00:00', '2018-10-04 07:00:00',
               '2018-10-04 08:00:00', '2018-10-04 09:00:00',
               '2018-10-04 10:00:00', '2018-10-04 11:00:00',
               '2018-10-04 12:00:00', '2018-10-04 13:00:00',
               '2018-10-04 14:00:00', '2018-10-04 15:00:00',
               '2018-10-04 16:00:00', '2018-10-04 17:00:00',
               '2018-10-04 18:00:00', '2018-10-04 19:00:00',
               '2018-10-04 20:00:00'],
              dtype='datetime64[ns]', freq='H')

In [78]:
df['dayR'] = df[ df['Time'] > datetime.time(12) ]

ValueError: cannot reindex from a duplicate axis

In [38]:
if df['Time'] < datetime.time(02:06:17.541000):
    print('night')

SyntaxError: invalid token (<ipython-input-38-87a332c7b476>, line 1)

In [ ]:
for index, row in df.iterrows():
    if df['Time'] < 20:01:
            

In [ ]:
df['dayR'] = df['Time'][(df['usertype'] == 'Subscriber') & (df['gender'] == 1)]
df['nightR'] = 2015 - df['birth year'][(df['usertype'] == 'Subscriber') & (df['gender'] == 2)]